In [ ]:
import matplotlib.pyplot as plt
import torch

import numpy as np

from gpytorch.mlls import VariationalELBO
from botorch.optim.fit import fit_gpytorch_torch

from volatilitygp.models import SingleTaskVariationalGP
from volatilitygp.likelihoods import BernoulliLikelihood

In [ ]:
import seaborn as sns
sns.set(font_scale=2.0, rc={"lines.linewidth":4} )

label_fs = 40

sns.set_style('whitegrid')
sns.set_palette("mako")

torch.__version__

label_fs = 40

In [ ]:
train_x = torch.tensor(np.loadtxt(
    "https://raw.githubusercontent.com/thangbui/streaming_sparse_gp/master/data/banana_train_x.txt",
    delimiter=","
))
train_y = torch.tensor(np.loadtxt(
    "https://raw.githubusercontent.com/thangbui/streaming_sparse_gp/master/data/banana_train_y.txt", delimiter=","
))
train_y[train_y==-1]=0

test_x = torch.tensor(np.loadtxt(
    "https://raw.githubusercontent.com/thangbui/streaming_sparse_gp/master/data/banana_test_x.txt",
    delimiter=","
))
test_y = torch.tensor(np.loadtxt(
    "https://raw.githubusercontent.com/thangbui/streaming_sparse_gp/master/data/banana_test_y.txt", delimiter=","
))
test_y[test_y==-1]=0

In [ ]:
ordering = train_x[:,0].sort()[1]
batch_1 = ordering[:100]
batch_2 = ordering[100:200]
batch_3 = ordering[200:300]
batch_4 = ordering[300:]

In [ ]:
batches = [batch_2, batch_3, batch_4]

In [ ]:
model = SingleTaskVariationalGP(
    init_points=train_x[batch_1],
    init_targets=train_y.view(-1)[batch_1],
    num_inducing=100,
    likelihood=BernoulliLikelihood(),
    use_piv_chol_init=True,
)

mll = VariationalELBO(model.likelihood, model, num_data = 200)

In [ ]:
fit_gpytorch_torch(mll);

In [ ]:
predicted_steps = []
pred_var = []

fant_model = model
for i in range(3):
    laplace_post = fant_model.posterior(test_x, observation_noise=True)
    prob = laplace_post.mean.exp() / (1 + laplace_post.mean.exp())
    
    f = plt.scatter(*test_x.t(), c=prob.detach(), s = 3, cmap="mako")
    plt.colorbar(f)
    plt.title("Step: " + str(i))
    plt.show()
    
    predicted_steps.append(prob.detach())
    
    # so we only include V(f)
    laplace_post = fant_model.posterior(test_x, observation_noise=False)
    pred_var.append(laplace_post.variance.detach())
    if i == 0:
        fant_model = fant_model.get_fantasy_model(train_x[batch_2], train_y[batch_2], targets_are_gaussian=False)
    else:
        targets = model.inducing_model().likelihood.newton_iteration(
            inputs=train_x[batches[i]], 
            targets=train_y[batches[i]], 
            covar=model.covar_module(train_x[batches[i]]),
        )
        noise = model.inducing_model().likelihood.neg_hessian_f(targets).diag()
        
        fant_model = fant_model.get_fantasy_model(train_x[batches[i]], targets, noise=noise.reciprocal())

laplace_post = fant_model.posterior(test_x, observation_noise=False)
prob = laplace_post.mean.exp() / (1 + laplace_post.mean.exp())
predicted_steps.append(prob.detach())
pred_var.append(laplace_post.variance.detach())
f = plt.scatter(*test_x.t(), c=prob.detach(), s = 3)
plt.colorbar(f)
plt.title("Step: " + str(3))
plt.show()

In [ ]:
i

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1,4, figsize=(20, 5), sharex=True, sharey=True)

f = ax[0].scatter(*test_x.t(), c=predicted_steps[0], s = 5, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[0])
f = ax[1].scatter(*test_x.t(), c=predicted_steps[1], s = 5, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[1])
f = ax[2].scatter(*test_x.t(), c=predicted_steps[2], s = 5, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[2])
f = ax[3].scatter(*test_x.t(), c=predicted_steps[3], s = 5, cmap="Blues", vmin=-0.5)
# fig.clim(0,1)
divider = make_axes_locatable(ax[3])
cax1 = divider.append_axes("right", size="5%", pad=0.05)
cbar = fig.colorbar(f, cax=cax1,boundaries=np.linspace(0,1,999), label="Probability")
cbar.set_ticks([0., 0.25, 0.5, 0.75, 1.0])
ax[0].set_xlim((-2.7,2.7))
ax[0].set_ylim((-2.7,2.7))
# ax[0].set_title("T = 100")
# ax[1].set_title("T = 200")
# ax[2].set_title("T = 300")
# ax[3].set_title("T = 400")

[ax[i].set_xlabel("x") for i in range(4)]
ax[0].set_ylabel("y")
plt.savefig("plots/bananas_probs.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 5), sharex=True, sharey=True)

f = ax[0].scatter(*train_x[batch_1].t(), c=train_y[batch_1], s = 50, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[0])
f = ax[1].scatter(*train_x[batch_1].t(), c=train_y[batch_1], s = 50, cmap="Blues", vmin=-0.5)
f = ax[1].scatter(*train_x[batch_2].t(), c=train_y[batch_2], s = 50, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[1])
f = ax[2].scatter(*train_x[batch_1].t(), c=train_y[batch_1], s = 50, cmap="Blues", vmin=-0.5)
f = ax[2].scatter(*train_x[batch_2].t(), c=train_y[batch_2], s = 50, cmap="Blues", vmin=-0.5)
f = ax[2].scatter(*train_x[batch_3].t(), c=train_y[batch_3], s = 50, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[2])
f = ax[3].scatter(*train_x.t(), c=train_y, s = 50, cmap="Blues", vmin=-0.5)
# fig.colorbar(f, ax=ax[3])
ax[0].set_xlim((-2.7,2.7))
ax[0].set_ylim((-2.7,2.7))

ax[0].set_title("T = 100")
ax[1].set_title("T = 200")
ax[2].set_title("T = 300")
ax[3].set_title("T = 400")

[ax[i].set_xlabel("x") for i in range(4)]
ax[0].set_ylabel("y")
# plt.savefig("plots/bananas_incremental.pdf", bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(*test_x.t(), c=test_y, cmap="mako", s=3)

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 5), sharex=True, sharey=True)

color_map = plt.cm.get_cmap('Blues')
reversed_color_map = color_map.reversed()
f = ax[0].scatter(*test_x.t(), c=pred_var[0], s = 3, cmap=reversed_color_map, vmax=5, vmin=0)
# fig.colorbar(f, ax=ax[0])
f = ax[1].scatter(*test_x.t(), c=pred_var[1], s = 3, cmap=reversed_color_map, vmax=5, vmin=0)
# fig.colorbar(f, ax=ax[1])
f = ax[2].scatter(*test_x.t(), c=pred_var[2], s = 3, cmap=reversed_color_map, vmax=5, vmin=0)
# fig.colorbar(f, ax=ax[2])
f = ax[3].scatter(*test_x.t(), c=pred_var[3], s = 3, cmap=reversed_color_map, vmax=5, vmin=0)
divider = make_axes_locatable(ax[3])
cax1 = divider.append_axes("right", size="5%", pad=0.05)
cbar = fig.colorbar(f, cax=cax1,boundaries=np.linspace(0,3.5,999), label="Variance")
cbar.set_ticks([0., 0.75, 1.5, 2.25, 3.0])
ax[0].set_xlim((-2.7,2.7))
ax[0].set_ylim((-2.7,2.7))
[ax[i].set_xlabel("x") for i in range(4)]
ax[0].set_ylabel("y")

plt.savefig("plots/bananas_variance.pdf", bbox_inches="tight")

In [ ]:
pred_var[3].max()